In [2]:
import argparse
import numpy as np
from gtransform_both import GraphAgent
from utils import *
import torch
import random
import time
import sys

path = 'GraphOOD-EERM/'
parser = argparse.ArgumentParser()
parser.add_argument('--gpu_id', type=int, default=0, help='gpu id')
parser.add_argument('--dataset', type=str, default='ogb-arxiv')
parser.add_argument('--epochs', type=int, default=50)
parser.add_argument('--hidden', type=int, default=32)
parser.add_argument('--weight_decay', type=float, default=5e-4)
parser.add_argument('--normalize_features', type=bool, default=True)
parser.add_argument('--seed', type=int, default=0, help='Random seed.')
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--lr_feat', type=float, default=0.001)
parser.add_argument('--nlayers', type=int, default=5)
parser.add_argument('--model', type=str, default='GCN')
parser.add_argument('--loss', type=str, default='LC')
parser.add_argument('--debug', type=int, default=1)
parser.add_argument('--ood', type=int, default=1)
parser.add_argument('--with_bn', type=int, default=1)
parser.add_argument('--lr_adj', type=float, default=0.1)
parser.add_argument('--ratio', type=float, default=0.1, help='budget B for changing graph structure')
parser.add_argument('--margin', type=float, default=-1)
parser.add_argument('--existing_space', type=int, default=1, help='enable removing edges from the graph')
parser.add_argument('--loop_adj', type=int, default=1, help='#loops for optimizing structure')
parser.add_argument('--loop_feat', type=int, default=4, help='#loops for optimizing features')
parser.add_argument('--test_val', type=int, default=0, help='set to 1 to evaluate performance on validation data')
parser.add_argument('--tune', type=int, default=0)
parser.add_argument('--finetune', type=int, default=0, help='whether to finetune the model')
parser.add_argument('--tent', type=int, default=0, help='use the Tent for finetuning (need to set finetune=1)')
parser.add_argument('--strategy', type=str, default='dropedge')
args = parser.parse_args("")

torch.cuda.set_device(args.gpu_id)

lr_feat = args.lr_feat; epochs = args.epochs; ratio = args.ratio; lr_adj = args.lr_adj
print('===========')
reset_args(args)
print(args)
if args.model == 'GAT':
    args.loop_adj = 0; args.loop_feat = args.epochs
if args.tune: # set args.tune to 1 to change the model hyperparameters
    args.lr_feat = lr_feat; args.epochs = epochs; args.ratio = ratio; args.lr_adj = lr_adj
if args.epochs == 2:
    args.loop_adj = 1; args.loop_feat = 1

path = 'GraphOOD-EERM/'
if args.dataset == 'elliptic':
    path = path + 'temp_elliptic'
    sys.path.append(path)
    from main_as_utils import datasets_tr, datasets_val, datasets_te
    data = [datasets_tr, datasets_val, datasets_te]
elif args.dataset == 'fb100':
    path = path + 'multigraph'
    sys.path.append(path)
    from main_as_utils_fb import datasets_tr, datasets_val, datasets_te
    data = [datasets_tr, datasets_val, datasets_te]
elif args.dataset == 'amazon-photo':
    path = path + 'synthetic'
    sys.path.append(path)
    from main_as_utils_photo import dataset_tr, dataset_val, datasets_te
    data = [dataset_tr, dataset_val, datasets_te]
else:
    if args.dataset == 'cora':
        path = path + 'synthetic'
    elif args.dataset == 'ogb-arxiv':
        path = path + 'temp_arxiv'
    elif args.dataset == 'twitch-e':
        path = path + 'multigraph'
    else:
        raise NotImplementedError
    sys.path.append(path)
    from main_as_utils import dataset_tr, dataset_val, datasets_te
    data = [dataset_tr, dataset_val, datasets_te]
# random seed setting
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

res = []
agent = GraphAgent(data, args)

Namespace(dataset='ogb-arxiv', debug=1, dropout=0, epochs=10, existing_space=1, finetune=0, gpu_id=0, hidden=32, loop_adj=1, loop_feat=4, loss='LC', lr=0.01, lr_adj=0.1, lr_feat=0.001, margin=-1, model='GCN', nlayers=5, normalize_features=True, ood=1, ratio=0.1, seed=0, strategy='dropedge', tent=0, test_val=0, tune=0, weight_decay=0, with_bn=1)
Namespace(K=3, T=1, beta=1.0, cached=False, cpu=False, data_dir='../data', dataset='ogb-arxiv', device=0, directed=False, display_step=100, dropout=0.0, epochs=500, gat_heads=2, gnn='gcn', gpr_alpha=0.1, hidden_channels=32, lp_alpha=0.1, lr=0.01, lr_a=0.005, method='erm', no_bn=False, num_layers=5, num_sample=5, rocauc=False, runs=5, sub_dataset='', weight_decay=0.0)
Train num nodes 17401 | target nodes 17401 | num classes 40 | num node feats 128
Val num nodes 41125 | target nodes 23724 | num classes 40 | num node feats 128
GCN(
  (layers): ModuleList(
    (0): GCNConv(128, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
    (3): GCNConv(3

In [4]:
# preliminary study
pred = F.softmax(agent.model.predict(data[0].graph['node_feat'].to(agent.device), data[0].graph['edge_index'].to(agent.device))).detach()
def entropy(pred):
    return -(pred * torch.log(pred)).sum(dim=1)
entropy_train = entropy(pred)

entropys = []
for data_test in data[2]:
    pred_temp = F.softmax(agent.model.predict(data_test.graph['node_feat'].to(agent.device), data_test.graph['edge_index'].to(agent.device))).detach()
    entropy_temp = entropy(pred_temp[data_test.test_mask] if hasattr(data_test, "test_mask") else pred_temp)
    entropy_temp[entropy_temp.isnan()] = 0
    entropys.append(entropy_temp)
# entropy_test = torch.stack(entropys)
print("Train mean entropy:", entropy_train.mean())
print("Test mean entropy:", [x.mean() for x in entropys])

Train mean entropy: tensor(1.7533, device='cuda:0')
Test mean entropy: [tensor(1.9408, device='cuda:0'), tensor(1.9609, device='cuda:0'), tensor(2.0648, device='cuda:0')]


/tmp/ipykernel_66792/3985302898.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(agent.model.predict(data[0].graph['node_feat'].to(agent.device), data[0].graph['edge_index'].to(agent.device))).detach()
/tmp/ipykernel_66792/3985302898.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred_temp = F.softmax(agent.model.predict(data_test.graph['node_feat'].to(agent.device), data_test.graph['edge_index'].to(agent.device))).detach()


In [11]:
# preliminary: loss value
data_temp = data[0]
feat, labels = data_temp.graph['node_feat'].to(agent.device), data_temp.label.to(agent.device)#.squeeze()
edge_index = data_temp.graph['edge_index'].to(agent.device)
agent.edge_index, agent.feat, agent.labels = edge_index, feat, labels
train_loss = agent.test_time_loss(agent.model, data_temp.graph['node_feat'].to(agent.device), data_temp.graph['edge_index'].to(agent.device), mode='eval')
print(train_loss)
test_losses = []
for data_temp in data[-1]:
    feat, labels = data_temp.graph['node_feat'].to(agent.device), data_temp.label.to(agent.device)#.squeeze()
    edge_index = data_temp.graph['edge_index'].to(agent.device)
    agent.edge_index, agent.feat, agent.labels = edge_index, feat, labels
    test_loss = agent.test_time_loss(agent.model, data_temp.graph['node_feat'].to(agent.device), data_temp.graph['edge_index'].to(agent.device), mode='eval')
    test_losses.append(test_loss.item())
print(test_losses)

tensor(-0.6272, device='cuda:0', grad_fn=<SubBackward0>)
[-0.6123806238174438, -0.5777472257614136, -0.5543553829193115]


/home/yy/miniconda3/envs/torch1121/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


In [5]:
accs = agent.evaluate(agent.model)

Test accs: [0.4077676746317051, 0.39058566382389104, 0.348785054420509]
flatten test: 0.37854279430345195


In [7]:
import scipy
print("Correlation:", scipy.stats.pearsonr(entropy_test.mean(dim=1).cpu(), accs))

Correlation: PearsonRResult(statistic=-0.7772486794327934, pvalue=0.02322074676564532)
